# Projet math-info lignes de niveau (Benjamin PIET et Antoine GAUCHOT)

In [ ]:
def find_seed (g, c=0.0, eps =2**(-26)):
    """
    Entrées :
        g : fonction à une variable ( ici, g(y) =f(0,y) )
        eps : la précison souhaitée
    Sortie :
        si la condition n'est pas vérifiée, on renvoie None
        sinon on renvoie un t tel que g(t) = c
    """
    if not( (g(0)<= c and g(1)>=c ) or ( g(0)>= c and g(1)<=c)) :    #on vérifie si la condition est vérifiée
        return(None)
    
    else :      #on cherche la solution par dichotomie
        a, b= 0, 1
    
        if ( g(a) - c > 0) :  #et donc d'après notre hypothèse g(0,b) - c < 0
            while ( abs (g(a) - c) > eps ) :
                t = (a+b)/2
                if ( g(t) > c):
                    a=t
                else :
                    b=t
    
        else :  #et donc d'après notre hypothèse g(0,b) - c > 0
            while ( abs (g(a) - c) > eps ) :
                t = (a+b)/2
                if ( g(t) < c):
                    a=t
                else :
                    b=t
        return(a)


def simple_contour (f, c=0.0, delta=0.01) :
    """
    """
    N = int( 1 / delta )    #
    X = np.array( [ delta * i for i in range(N) ] )
    Y= []
    for i in range (N) :
        def g(y) :
            return( f( X[i],y ) ) 
        t = find_seed (g, c) 
        if t != None :
            Y.append(t)
        
    X=X[:len(Y)]
    return (X,Y)

In [ ]:
f00 = -1
f01 = +1.2
f10 = +1
f11 = -0.8
def f(x, y):
    fx0 = f00 * (1 - x) + f10 * x
    fx1 = f01 * (1 - x) + f11 * x
    return fx0 * (1 - y) + fx1 * y

X,Y=simple_contour_upgrd(f, c=0)
plt.plot(X,Y)
plt.show()


In [ ]:
def newton (h, X0, eps=2**(-26)) :
    """
    renvoie X tel que h(X) = 0
    """
    X=np.array(X0)
    def J_h(x,y):
        j=autograd.jacobian
        return np.c_[j(h,0)(x,y),j(h,1)(x,y)]
    while abs(h(X[0],X[1]).all())>eps:
        print(X[0],X[1])
        print(J_h(X[0],X[1]))
        J_inv=np.linalg.inv(J_h(X[0],X[1]))
        X=X-J_inv.dot(h(X[0],X[1]))
    return X

def simple_contour_upgrd(f, c=0.0,delta=0.01):
    les_x=[0.0]
    les_y=[]
    def g(y) :
            return( f( les_x[0],y ) ) 
    t = find_seed (g, c)  
    if t != None :
        les_y.append(t)
        
        #on a notre premier point de notre courbe. on va mtn faire la propagation
    while 0<=les_x[-1]<=1 and 0<=les_y[-1]<=1:
        xi=les_x[-1]
        yi=les_y[-1]
        Xi=np.array([xi,yi])
        def grad_f(x,y):
            g=autograd.grad
            return np.r_[g(f,0)(x,y),g(f,1)(x,y)]
        grad=grad_f(xi,yi)
        grad_orth=np.array([grad[1],-grad[0]])
        if grad_orth[1]<0:
            grad_orth[0],grad_orth[1]=-grad_orth[0],-grad_orth[1]
        Xg=Xi+delta*grad_orth
              
        
        def h(x,y):
            return np.array([[(x-xi)**2+(y-yi)**2-delta**2],[f(x,y)-c]])
        
        x,y=newton(h,Xg)
        les_x.append(x)
        les_y.append(y)
    return les_x,les_y